In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000647639s, CPU 0.000612s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0116269s, CPU 0.011611s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:AxiSymmetricHeatConduction)

Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, -1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [5]:
T0 = FEM.initialTemperature(problem, "body", T=0)

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [7]:
FEM.initialTemperature!(T0, "left", T=0)
T0

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [8]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

ScalarField
[0.0; -0.39269908169872364; … ; 0.0; 0.0;;]

In [10]:
λmax = FEM.largestEigenValue(K, C, support=[supp])

28621.75593572514

In [11]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200, 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

ScalarField
[0.0 -7.820103104728526e-7 … -3.662078497777416e48 6.945686833181558e48; 0.0 -0.006401883635274431 … 5.570189249129069e49 -1.0448310119983978e50; … ; 0.0 -3.6520953987030477e-6 … 5.236610535008946e49 -9.898086369255968e49; 0.0 -0.0004684874133875409 … -1.1916548660429377e50 2.24141050597474e50]

In [12]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 15, 0.01, ϑ=ϑ)

ScalarField
[0.0 -0.0018291835307356716 … -0.0731978447680257 -0.07926396338954089; 0.0 -0.012080777021853104 … -0.08439814951142412 -0.09046417676683555; … ; 0.0 -0.0022273680058058605 … -0.07401791105019537 -0.08008403271065181; 0.0 -0.006162373478812534 … -0.07951894155007143 -0.08558505268197066]

In [13]:
FEM.showDoFResults(T, :T, visible=true)

1

In [14]:
q1 = FEM.solveHeatFlux(T)

VectorField
[[0.0 0.923154948393539 … 0.8344492326936388 0.8344756336564385; 0.0 -0.0043794765221116805 … -0.0015890442015620287 -0.0015899733469176835; … ; 0.0 0.923154948393539 … 0.8344492326936388 0.8344756336564385; 0.0 -0.0043794765221116805 … -0.0015890442015620287 -0.0015899733469176835], [0.0 0.9245547216957027 … 0.8344592127736999 0.8344853235180316; 0.0 -0.006511492347931538 … -0.0016042450120024032 -0.0016047321212403487; … ; 0.0 0.9245547216957027 … 0.8344592127736999 0.8344853235180316; 0.0 -0.006511492347931538 … -0.0016042450120024032 -0.0016047321212403487], [0.0 0.9022995507719795 … 0.8266782209694262 0.8267047217511553; 0.0 0.02147943260463449 … 0.006878535276043235 0.00687543773647839; … ; 0.0 0.9022995507719795 … 0.8266782209694262 0.8267047217511553; 0.0 0.02147943260463449 … 0.006878535276043235 0.00687543773647839], [0.0 0.09513893618660935 … 0.1933940183792443 0.19339474807050172; 0.0 0.008627443071214536 … 0.016110744727260984 0.016110807575050456; … ; 0.0 0.09

In [15]:
FEM.probe(q1, 0.5, 0.5, 0, step=10)

2-element Vector{Float64}:
 0.4983368372
 0.0163861838

In [16]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [17]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

VectorField
[0.0 0.03442506670410861 … 0.07355170378979897 0.07355195494593486; 0.0 -0.0009003386994381858 … -0.0019373920501492697 -0.001937399768486489; … ; 0.0 0.7197059527214783 … 0.7481744341402025 0.7481864797904141; 0.0 0.0013508207147682784 … 0.0003182058194273196 0.0003179972474035009]

In [18]:
FEM.showDoFResults(q2, :qvec)

3

In [19]:
FEM.probe(q2, 0.5, 0.5, 0, step=10)

2-element Vector{Float64}:
  0.5004364459
 -8.82239e-5

In [20]:
q3 = FEM.elementsToNodes(q1)

VectorField
[0.0 0.03442506670410861 … 0.07355170378979897 0.07355195494593486; 0.0 -0.0009003386994381858 … -0.0019373920501492697 -0.001937399768486489; … ; 0.0 0.7197059527214783 … 0.7481744341402025 0.7481864797904141; 0.0 0.0013508207147682784 … 0.0003182058194273196 0.0003179972474035009]

In [21]:
FEM.showDoFResults(q3, :qvec)

4

In [22]:
err1 = FEM.fieldError(q1)

VectorField
[0.0 0.002864137630585021 … 0.006112670548877597 0.006112692203350811; 0.0 0.0028641376305850235 … 0.006112670548875723 0.006112692203350187; … ; 0.0 0.11226528492857941 … 0.05141053300883799 0.051422089495732425; 0.0 0.04824565202194338 … 0.022733966173658856 0.02273857297453997]

In [23]:
FEM.showDoFResults(err1, :vector)

5

In [24]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [25]:
gmsh.finalize()